# ***Informacion del dataset***

In [1]:

%%html
<style> table { display: inline-block  }</style>

https://www.kaggle.com/datasets/riteshahlawat/covid19-mexico-patient-health-dataset



|col|0|1|97-98|
|---|---|---|---|
|sex|F|M|?|
|hospitalized|no|si|?|
|intubated|no|si|?|
|pneumonia|no|si|?|
|age|no|si|?|
|pregnant|no|si|?|
|diabetes|no|si|?|
|copd|no|si|?|
|asthma|no|si|?|
|immunosuppression|no|si|?|
|hypertension|no|si|?|
|other_diseases|no|si|?|
|cardiovascular|no|si|?|
|obesity|no|si|?|
|chronic_kidney_failure|no|si|?|
|smoker|no|si|?|
|another_case|no|si|?|
|outcome|no|si|?|
|icu|no|si|?|
|dead|no|si|?|

# **1. Importación de *modules***

In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from numpy.ma.core import count

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings("ignore")

import funciones as f

# **2. Importación del dataset *Patient.csv***


In [3]:
data = pd.read_csv("../data/patient.csv", sep = ",", low_memory=False) 
f._get_info(data)

# ¿No es mejor hacer un data.info()?

FileNotFoundError: [Errno 2] No such file or directory: '../data/patient.csv'

# **3. Tratamiento de los datos**
Limpieza e imputaciones

### 3.1. Dropeo de columnas no informativas

In [ ]:
#Dropeo icu, intubated, death_date
data = data.drop(['icu','intubated','death_date'], axis=1)
print(data.columns)
#f._get_info(data)
data.shape

In [ ]:
#99 a null y dropeo
cols=data.columns.difference(['age'])

df=(data[cols] == 99)
ix=df[df.any(axis=1)].index
print(ix)
data.drop(df.index[ix], axis=0, inplace=True)
data.shape

# Propongo hacer un código más simple, como:
# data = data.apply(lambda x: x.replace(99, np.NaN) if x.name != 'age' else x).dropna()
# Se puede chequear el resultado con data.info()

### 3.2. Renombre de columnas ambiguas
Por ejemplo, patient_type a inpatient, outcome a covid, etc

In [ ]:
data.rename(columns={"patient_type": "inpatient", "outcome": "covid"},inplace=True)
data.columns

Matriz de correlación para regresión logística

In [ ]:
f._print_matriz_correlacion(data)

### 3.3 Conversión de datos 

In [ ]:
# Para predictivas: 1 -> Y, 2 -> N, 
cols = data.columns.difference(['age', 'sex', 'inpatient'])

data[cols] = data[cols].replace([1, 2], ['Y', 'N'])

# en sex: 1 -> F y 2-> M
data['sex'] = data['sex'].replace([1, 2], ['F', 'M'])

# en pregnant : 98 -> N, 97 -> NA
data['pregnant'] = data['pregnant'].replace([98, 97], ['N', 'N'])

# outcome: 3 -> NA
data['covid'] = data['covid'].replace([3], ['NOA'])

# resto del dataset: 98 -> NotA
data[cols] = data[cols].replace(98, 'NOA')

## Para target (inpatient): 1 -> 0, 2 -> 1
data['inpatient'] = data['inpatient'].replace([1, 2], [0, 1])

data.head(5)

In [ ]:
cols_plt = data.columns.difference(['age', 'sex', 'inpatient'])

# Creamos un nuevo dataframe para hacer el gráfico, sólo con dos columnas, % de nulos y no nulos por variable
val_yes = data[cols_plt].apply(lambda x: (x=='Y').sum() / data.shape[0] * 100)
val_not = data[cols_plt].apply(lambda x: (x=='N').sum() / data.shape[0] * 100)
val_na = data[cols_plt].apply(lambda x: (x=='NOA').sum() / data.shape[0] * 100)
#val_1 = data.apply(lambda x: (x=='1').count() / data.shape[0] * 100)
#sry_null = pd.DataFrame({'Y': val_yes, 'N': val_not ,'NA': val_na ,'1': val_1})
sry_null = pd.DataFrame({'Y': val_yes, 'N': val_not ,'NotA': val_na })

sns.set(style = 'dark')

fig, ax = plt.figure(figsize = (13, 6)), plt.axes()
sry_null.sort_values(by = 'N').plot(kind = 'bar', stacked = True, color = ['#0CA4A5', '#BB7E5D' , '#CC7EEE','#0CccA5'], ax = ax)

ax.set(xlabel = 'Columnas', ylabel = '% ', title = 'Datos ', ylim = (0, 130))
ax.legend(loc = 'upper left')
# Guardo la imagen
#en colab
#images_dir = '/content/drive/My Drive/DSDH/TP1/Imagenes/null_barplot.png'
#en jupyter
plt.show()
#plt.savefig("null_barplot.png", dpi = 150, bbox_inches = 'tight', pad_inches = 0.1)

### 3.5. Tabla pivot por inpatient

In [ ]:
#creo un total count inpatient para ponderar resultados
total_inpatient = data['inpatient'].count()

#by sex
round(pd.pivot_table(data, values = ['inpatient'], index = ['sex'],
                     aggfunc=['count',(lambda x: count(x)/total_inpatient*100)]))

In [ ]:
f._summary(data)

### 3.6. Creación de categorías a partir de edad

In [ ]:
#creo categorias para edad por décadas 
data = data.assign(agegroup=pd.cut(data['age'], bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120], right=False, labels=["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80-89","90-99", "100-120"]))
#pruedo crear categorías para edad por etapa de la vida
# como definimos? 0-18 niños, 18 a 30 jovenes, 30 a 60 Adultos, 60 a 80 adultos mayores, más de 80 ancianos?

f._summary(data)

### 3.7. Creación de dummies

In [ ]:
#elimino age para no tener dummies
data.drop(["age"], axis=1, inplace=True)

#creo variables dummies de todas las columnas menos la target
data = pd.get_dummies(data=data, columns= data.columns.difference(["inpatient"]), drop_first = True)

In [ ]:
data.columns

# **4. Exportación del dataset resultante a un nuevo *.csv***

In [ ]:
# para exportar resultados
data.to_csv(r'../data/patient_covid.csv', index = False, header=True)
data.shape
data